# Exporting American Movie Box Office Hits 

### boxofficemojo.com data acquisition

In [110]:
from bs4 import BeautifulSoup as bs
import requests
import pandas as pd
import re
import dateutil.parser
import time
import random


In [125]:
# NEW

# adaptation genre has nine pages, need to scrape information from all nine pages

# WISHLIST
# url_base = 'https://www.boxofficemojo.com/genre/'
offset = 0
base_url = 'https://www.boxofficemojo.com/genre/sg1625223425/?offset='

movies_urls = []

for url in range(10):
    movies_urls.append(base_url + str(offset))
    offset = offset + 100

In [139]:
# code works on ONE page at a time
# request html for index page genre adaptation (n=964):
def scrape_movies(url, movies_data):
#     url = 'https://www.boxofficemojo.com/genre/sg1625223425/?ref_=bo_gs_table_10'
    response = requests.get(url)
    status = response.status_code
    if status == 200:
        print('Status is 200')
        page = response.text
        soup = bs(page, "lxml")
        table = soup.find("table")
        rows = [row for row in table.find_all('tr')] 
        for row in rows:
            movies_data.append(row)
    else:
        print(f"Oops! Received status code {status}")
        return None
    
movies_data = []

for movie_url in movies_urls:
    scrape_movies(movie_url, movies_data)
    
print(movies_data)


Status is 200
Status is 200
Status is 200
Status is 200
Status is 200
Status is 200
Status is 200
Status is 200
Status is 200
Status is 200
[<tr><th class="a-text-right mojo-field-type-rank mojo-sortable-column a-nowrap"><span title="Rank">Rank</span>
</th><th class="a-text-left mojo-field-type-title mojo-sortable-column a-nowrap"><span title="Title">Title</span>
</th><th class="a-text-right mojo-field-type-money mojo-sort-column mojo-sortable-column mojo-estimatable a-nowrap"><a class="a-link-normal a-nowrap" href="?offset=0&amp;sort=grossToDate&amp;sortDir=asc&amp;ref_=bo_ge__resort#table" title="Lifetime Gross"><span class="a-color-state">Lifetime Gross</span><span class="a-letter-space"></span><span class="icon aok-relative"><i class="a-icon a-icon-expand" role="presentation"></i></span></a></th><th class="a-text-right mojo-field-type-positive_integer mojo-sortable-column a-nowrap"><a class="a-link-normal a-nowrap" href="?offset=0&amp;sort=maxNumTheaters&amp;ref_=bo_ge__resort#tabl

In [145]:
# print(movies_data)
# type(movies_data[0])
# print(movies_data[1].prettify())

movies_data.pop(0)

movies_dict = {}


for row in movies_data:
    items = row.find_all('td')
    print(items)
    link = items[1].find('a')
    title, url = link.text, link['href']
    movies_dict[title] = [url] + [i.text for i in items]

print(movies_dict)


[<td class="a-text-right mojo-header-column mojo-truncate mojo-field-type-rank">2</td>, <td class="a-text-left mojo-field-type-title"><a class="a-link-normal" href="/title/tt10872600/?ref_=bo_ge_table_2">Spider-Man: No Way Home</a></td>, <td class="a-text-right mojo-field-type-money mojo-sort-column mojo-estimatable">$789,170,954</td>, <td class="a-text-right mojo-field-type-positive_integer">4,336</td>, <td class="a-text-right mojo-field-type-money">$260,138,569</td>, <td class="a-text-right mojo-field-type-positive_integer">4,336</td>, <td class="a-text-left mojo-field-type-date a-nowrap"><a class="a-link-normal" href="/date/2021-12-17/?ref_=bo_ge_table_2">Dec 17, 2021</a></td>, <td class="a-text-left mojo-field-type-release_studios"><a class="a-link-normal" href="https://pro.imdb.com/company/co0026545/boxoffice/?view=releases&amp;ref_=mojo_ge_table_2&amp;rf=mojo_ge_table_2" rel="noopener" target="_blank">Sony Pictures Entertainment (SPE)<svg class="mojo-new-window-svg" viewbox="0 0 

IndexError: list index out of range

# `movies_dict`
Get link stub from index page.

In [114]:
# scrape data from each row, including the link stub 

rows = [row for row in table.find_all('tr')]  # tr tag is for rows

movies_dict = {}

for row in rows[1:2]:
    items = row.find_all('td')
    link = items[1].find('a')
    title, url = link.text, link['href']
    movies_dict[title] = [url] + [i.text for i in items]
movies_dict


{'Avengers: Endgame': ['/title/tt4154796/?ref_=bo_ge_table_1',
  '1',
  'Avengers: Endgame',
  '$858,373,000',
  '4,662',
  '$357,115,007',
  '4,662',
  'Apr 26, 2019',
  'Walt Disney Studios Motion Pictures\n\n']}

# `adaptation_movies_df` 

In [115]:
# df using movies dict from above, including link_stub

adaptation_movies_df = pd.DataFrame(movies_dict).T  #transpose
adaptation_movies_df.columns = ['link_stub', 'rank', 'movie_title', 
                    'lifetime_gross', 'max_theaters', 'opening_weekend', 
                    'opening_theathers', 'release_date', 'distributor']
adaptation_movies_df.head()


,link_stub,rank,movie_title,lifetime_gross,max_theaters,opening_weekend,opening_theathers,release_date,distributor
Avengers: Endgame,/title/tt4154796/?ref_=bo_ge_table_1,1,Avengers: Endgame,"$858,373,000","4,662","$357,115,007","4,662","Apr 26, 2019",Walt Disney Studios Motion Pictures\n\n


# helper functions
# `money_to_int()`  `runtime_to_minutes()`  `to_date()`

In [116]:
# helper functions parse strings into appropriate data types

def money_to_int(moneystring):
    moneystring = moneystring.replace('$', '').replace(',', '')
    return int(moneystring)

def runtime_to_minutes(runtimestring):
    runtime = runtimestring.split()
    try:
        minutes = int(runtime[0])*60 + int(runtime[2])
        return minutes
    except:
        return None

def to_date(datestring):
    date = dateutil.parser.parse(datestring)
    return date


# `get_movie_value()`

In [47]:
# function to scrape values from each movie page, will pass field_name in next function

def get_movie_value(soup, field_name):
 
    obj = soup.find(text=re.compile(field_name))
    
    if not obj: 
        return None
    
    next_element = obj.findNext()
    
    if next_element:
        return next_element.text 
    else:
        return None

# `get_movie_dict()`

In [48]:
# create movie dict to collect values from get_movie_value function from above

def get_movie_dict(link):
    
    base_url = 'https://www.boxofficemojo.com'
    
    # create full url to scrape
    url = base_url + link
    
    # request HTML and parse
    response = requests.get(url)
    status = response.status_code
    if status == 200:
        page = response.text
        soup = bs(page, "lxml")
    else:
        print(f"Oops! Received status code {status}")
        
    headers = ['movie_title', 
               'domestic_total_gross',
               'international_total_gross',
               'worldwide_total_gross',
               'domestic_opening',
               'budget',
               'release_date',
               'runtime',
               'distributor',
               'genres',
               'rating'
              ]

    #get title
    title_string = soup.find('title').text
    title = title_string.split('-')[0].strip()
    print(title) # make sure scraper is working

    '''
    Get GROSS VALUES from performance summary table on left side of each movie page
    '''
    # domestic $$$
    try:     
        raw_domestic_total_gross = (soup.find(class_='mojo-performance-summary-table')
                                        .find_all('span', class_='money')[0].text)
        # convert raw to int       
        domestic_total_gross = money_to_int(raw_domestic_total_gross)
    except:
        domestic_total_gross= 0
    
    # international $$$
    try: 
        raw_international_total_gross = (soup.find(class_='mojo-performance-summary-table')
                                        .find_all('span', class_='money')[1].text)
        #convert raw to int    
        international_total_gross = money_to_int(raw_international_total_gross)
    except:
         international_total_gross = 0

    # worldwide $$$
    try: 
        raw_worldwide_total_gross = (soup.find(class_='mojo-performance-summary-table')
                                        .find_all('span', class_='money')[2]
                                        .text)
        # convert raw to int
        worldwide_total_gross = money_to_int(raw_worldwide_total_gross)

    except:
        worldwide_total_gross = 0

    '''
    Get MOVIE DETAILS from summary details table in center of each movie page
    '''    
    # domestic opening $$$ 
    try: 
        raw_domestic_opening = get_movie_value(soup,'Domestic Opening')
        
        # convert to int
        domestic_opening = money_to_int(raw_domestic_opening)
    except:
        domestic_opening = 0
    
    # budget $$$
    try: 
        raw_budget = get_movie_value(soup,'Budget')
        
        # convert budget to int
        budget = money_to_int(raw_budget)
    except:
        budget = 0
    
    # release date
    try: 
        raw_release_date = get_movie_value(soup,'Release Date').split('\n')[0]
        # convert date to dt
        release_date = to_date(raw_release_date)
    except:
        release_date = 0

    # runtime 
    try: 
        raw_runtime = get_movie_value(soup,'Running')
        # convert runtime to minutes
        runtime = runtime_to_minutes(raw_runtime)
    except:
        runtime = 0 
    
    # distributor
    try:  
        raw_distributor = get_movie_value(soup,'Distributor')
        # clean up 
        distributor = raw_distributor.replace('See full company information', '').replace('\n\n', '')
    except:
        distributor = 'NA' 

    # genres
    try: 
        genres = get_movie_value(soup,'Genres').replace('\n    \n        ', ', ')
    except:
        genres = 'NA'

    # rating    
    try: 
        rating = get_movie_value(soup,'MPAA')
    except:
        rating = 'NA'
    
    # movie dictionary
    movie_dict = dict(zip(headers, [title,
                                    domestic_total_gross,
                                    international_total_gross,
                                    worldwide_total_gross,
                                    domestic_opening,
                                    budget,
                                    release_date,
                                    runtime,
                                    distributor,
                                    genres,
                                    rating]))
    
#     print(movie_dict)

    return movie_dict

# `adaptation_movie_page_info_list`

In [49]:
# loop to pass link using get_movie_dict() function from above

adaptation_movie_page_info_list = []

for link in adaptation_movies_df.link_stub:
    adaptation_movie_page_info_list.append(get_movie_dict(link))

Avengers: Endgame
Spider
Black Panther
Avengers: Infinity War
The Avengers
The Lion King
The Dark Knight
Beauty and the Beast
Avengers: Age of Ultron
The Dark Knight Rises
Captain Marvel
The Hunger Games: Catching Fire
Wonder Woman
Iron Man 3
Captain America: Civil War
The Hunger Games
Spider
Transformers: Revenge of the Fallen
Spider


In [50]:
adaptation_movie_page_info_list

[{'movie_title': 'Avengers: Endgame',
  'domestic_total_gross': 858373000,
  'international_total_gross': 1939128328,
  'worldwide_total_gross': 2797501328,
  'domestic_opening': 357115007,
  'budget': 356000000,
  'release_date': datetime.datetime(2019, 4, 24, 0, 0),
  'runtime': 181,
  'distributor': 'Walt Disney Studios Motion Pictures',
  'genres': 'Action, Adventure, Drama, Sci-Fi',
  'rating': 'PG-13'},
 {'movie_title': 'Spider',
  'domestic_total_gross': 789170954,
  'international_total_gross': 1080000000,
  'worldwide_total_gross': 1869170954,
  'domestic_opening': 260138569,
  'budget': 0,
  'release_date': datetime.datetime(2021, 12, 15, 0, 0),
  'runtime': 148,
  'distributor': 'Sony Pictures Entertainment (SPE)',
  'genres': 'Action, Adventure, Fantasy, Sci-Fi',
  'rating': 'PG-13'},
 {'movie_title': 'Black Panther',
  'domestic_total_gross': 700426566,
  'international_total_gross': 647171407,
  'worldwide_total_gross': 1347597973,
  'domestic_opening': 202003951,
  'budg

# `adaptation_movie_info_df` from `adaptation_movie_page_info_list`

In [51]:
# #convert list of dict to df

adaptation_movie_info_df = pd.DataFrame(adaptation_movie_page_info_list)  
adaptation_movie_info_df.set_index('movie_title', inplace=True)

adaptation_movie_info_df.info()


<class 'pandas.core.frame.DataFrame'>
Index: 19 entries, Avengers: Endgame to Spider
Data columns (total 10 columns):
 #   Column                     Non-Null Count  Dtype         
---  ------                     --------------  -----         
 0   domestic_total_gross       19 non-null     int64         
 1   international_total_gross  19 non-null     int64         
 2   worldwide_total_gross      19 non-null     int64         
 3   domestic_opening           19 non-null     int64         
 4   budget                     19 non-null     int64         
 5   release_date               19 non-null     datetime64[ns]
 6   runtime                    19 non-null     int64         
 7   distributor                19 non-null     object        
 8   genres                     19 non-null     object        
 9   rating                     19 non-null     object        
dtypes: datetime64[ns](1), int64(6), object(3)
memory usage: 1.6+ KB


In [52]:
adaptation_movie_page_info_list

[{'movie_title': 'Avengers: Endgame',
  'domestic_total_gross': 858373000,
  'international_total_gross': 1939128328,
  'worldwide_total_gross': 2797501328,
  'domestic_opening': 357115007,
  'budget': 356000000,
  'release_date': datetime.datetime(2019, 4, 24, 0, 0),
  'runtime': 181,
  'distributor': 'Walt Disney Studios Motion Pictures',
  'genres': 'Action, Adventure, Drama, Sci-Fi',
  'rating': 'PG-13'},
 {'movie_title': 'Spider',
  'domestic_total_gross': 789170954,
  'international_total_gross': 1080000000,
  'worldwide_total_gross': 1869170954,
  'domestic_opening': 260138569,
  'budget': 0,
  'release_date': datetime.datetime(2021, 12, 15, 0, 0),
  'runtime': 148,
  'distributor': 'Sony Pictures Entertainment (SPE)',
  'genres': 'Action, Adventure, Fantasy, Sci-Fi',
  'rating': 'PG-13'},
 {'movie_title': 'Black Panther',
  'domestic_total_gross': 700426566,
  'international_total_gross': 647171407,
  'worldwide_total_gross': 1347597973,
  'domestic_opening': 202003951,
  'budg

# 4. Merge `adaptation_movies_df` and `adaptation_movie_info_df` dfs 
# `g_movies` df

In [53]:
# match this back up with the movie information collected from the table by merging  dataframes
adaptation_movies_df = adaptation_movies_df.merge(adaptation_movie_info_df, left_index=True, right_index=True)
adaptation_movies_df

,link_stub,rank,movie_title,lifetime_gross,max_theaters,opening_weekend,opening_theathers,release_date_x,distributor_x,domestic_total_gross,international_total_gross,worldwide_total_gross,domestic_opening,budget,release_date_y,runtime,distributor_y,genres,rating
Avengers: Age of Ultron,/title/tt2395427/?ref_=bo_ge_table_9,9,Avengers: Age of Ultron,"$459,005,868","4,276","$191,271,109","4,276","May 1, 2015",Walt Disney Studios Motion Pictures\n\n,459005868,943803672,1402809540,191271109,250000000,2015-04-22,141,Walt Disney Studios Motion Pictures,"Action, Adventure, Sci-Fi",PG-13
Avengers: Endgame,/title/tt4154796/?ref_=bo_ge_table_1,1,Avengers: Endgame,"$858,373,000","4,662","$357,115,007","4,662","Apr 26, 2019",Walt Disney Studios Motion Pictures\n\n,858373000,1939128328,2797501328,357115007,356000000,2019-04-24,181,Walt Disney Studios Motion Pictures,"Action, Adventure, Drama, Sci-Fi",PG-13
Avengers: Infinity War,/title/tt4154756/?ref_=bo_ge_table_4,4,Avengers: Infinity War,"$678,815,482","4,474","$257,698,183","4,474","Apr 27, 2018",Walt Disney Studios Motion Pictures\n\n,678815482,1369544272,2048359754,257698183,0,2018-04-25,149,Walt Disney Studios Motion Pictures,"Action, Adventure, Sci-Fi",PG-13
Beauty and the Beast,/title/tt2771200/?ref_=bo_ge_table_8,8,Beauty and the Beast,"$504,014,165","4,210","$174,750,616","4,210","Mar 17, 2017",Walt Disney Studios Motion Pictures\n\n,504481165,769095055,1273576220,174750616,160000000,2017-03-16,129,Walt Disney Studios Motion Pictures,"Adventure, Family, Fantasy, Musical, Romance",PG
Black Panther,/title/tt1825683/?ref_=bo_ge_table_3,3,Black Panther,"$700,059,566","4,084","$202,003,951","4,020","Feb 16, 2018",Walt Disney Studios Motion Pictures\n\n,700426566,647171407,1347597973,202003951,0,2018-02-13,134,Walt Disney Studios Motion Pictures,"Action, Adventure, Sci-Fi",PG-13
Captain America: Civil War,/title/tt3498820/?ref_=bo_ge_table_15,15,Captain America: Civil War,"$408,084,349","4,226","$179,139,142","4,226","May 6, 2016",Walt Disney Studios Motion Pictures\n\n,408084349,745253147,1153337496,179139142,250000000,2016-04-27,147,Walt Disney Studios Motion Pictures,"Action, Adventure, Sci-Fi",PG-13
Captain Marvel,/title/tt4154664/?ref_=bo_ge_table_11,11,Captain Marvel,"$426,829,839","4,310","$153,433,423","4,310","Mar 8, 2019",Walt Disney Studios Motion Pictures\n\n,426829839,701633133,1128462972,153433423,160000000,2019-03-06,123,Walt Disney Studios Motion Pictures,"Action, Adventure, Sci-Fi",PG-13
Iron Man 3,/title/tt1300854/?ref_=bo_ge_table_14,14,Iron Man 3,"$409,013,994","4,253","$174,144,585","4,253","May 3, 2013",Walt Disney Studios Motion Pictures\n\n,409013994,805797258,1214811252,174144585,200000000,2013-04-24,130,Walt Disney Studios Motion Pictures,"Action, Adventure, Sci-Fi",PG-13
The Avengers,/title/tt0848228/?ref_=bo_ge_table_5,5,The Avengers,"$623,357,910","4,349","$207,438,708","4,349","May 4, 2012",Walt Disney Studios Motion Pictures\n\n,623357910,895457605,1518815515,207438708,220000000,2012-04-25,143,Walt Disney Studios Motion Pictures,"Action, Adventure, Sci-Fi",PG-13
The Dark Knight,/title/tt0468569/?ref_=bo_ge_table_7,7,The Dark Knight,"$533,345,358","4,366","$158,411,483","4,366","Jul 18, 2008",Warner Bros.\n\n,534987076,471115201,1006102277,158411483,185000000,2008-07-16,152,Warner Bros.,"Action, Crime, Drama, Thriller",PG-13


In [54]:
adaptation_movies_df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 16 entries, Avengers: Age of Ultron to Wonder Woman
Data columns (total 19 columns):
 #   Column                     Non-Null Count  Dtype         
---  ------                     --------------  -----         
 0   link_stub                  16 non-null     object        
 1   rank                       16 non-null     object        
 2   movie_title                16 non-null     object        
 3   lifetime_gross             16 non-null     object        
 4   max_theaters               16 non-null     object        
 5   opening_weekend            16 non-null     object        
 6   opening_theathers          16 non-null     object        
 7   release_date_x             16 non-null     object        
 8   distributor_x              16 non-null     object        
 9   domestic_total_gross       16 non-null     int64         
 10  international_total_gross  16 non-null     int64         
 11  worldwide_total_gross      16 non-null     int

In [55]:
# picke df 
adaptation_movies_df1 = adaptation_movies_df
adaptation_movies_df1.to_pickle('adaptation_movies_df1.pkl')


In [56]:
# save to csv
adaptation_movies_df1.to_csv(r'/Users/sandraparedes/Dropbox/Mac/Downloads/adaptation_movies_df1.csv', index=False)

In [ ]:
# pd.read_csv('g_movies_p1_df.csv')  

In [ ]:
# # read pickle file as dataframe
# g_movies_df = pd.read_pickle('g_movies_df.pkl')
# g_movies_df